In [1]:
import datetime
import math
import os
import pickle
import random

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [2]:
def read_in_data(text_file):

    data_output = []
    with open(text_file, "r") as file1:
        data_list = file1.readlines()
    for line in data_list:
        token_list = line.split()
        data_output.append(token_list)
    
    return data_output

In [33]:
training_data = read_in_data("../data/english-french_small/dev.en")

training_data = training_data[:5]

In [34]:
def create_vocabulary(training_set):

    vocabulary = []
    for sentence in training_set:
        for word in sentence:
            if word not in vocabulary:
                vocabulary.append(word)
                
    w2i = dict()
    i2w = dict()
    for idx, word in enumerate(vocabulary):
        i2w[idx] = word
        w2i[word] = idx
    
    return w2i, i2w

In [35]:
w2i, i2w = create_vocabulary(training_data)
V = len(w2i.keys())

In [36]:
print(len(w2i.keys()))

74


In [37]:
def generate_skipgram(sentence, context_window_size):

    skipgram_array = []
    for idx, word in enumerate(sentence):
        context_set = []
        window_size = context_window_size
        for index in range(max(idx - window_size, 0), min(len(sentence), idx + window_size + 1)):
            if index != idx:
                context_set.append([word, sentence[index]])
        skipgram_array.append(context_set)

    return skipgram_array

In [38]:
WINDOW_SIZE = 2
# Training data :
context_data = [generate_skipgram(sentence, WINDOW_SIZE) for sentence in training_data]

In [39]:
# flat_list = []


# for sublist in context_data:
#     for item_1 in sublist:
#         for item_2
#         flat_list.append(item)
        
    

print(len(context_data[0]))
print(len(training_data[0]))


print(training_data[0])
print("======")
print(context_data[0])


28
28
['each', 'of', 'them', 'is', 'very', 'complex', ',', 'but', 'the', 'link', 'between', 'the', 'two', 'is', 'even', 'more', 'complex', 'which', 'makes', 'the', 'whole', 'situation', 'for', 'most', 'people', 'understandably', 'confusing', '.']
[[['each', 'of'], ['each', 'them']], [['of', 'each'], ['of', 'them'], ['of', 'is']], [['them', 'each'], ['them', 'of'], ['them', 'is'], ['them', 'very']], [['is', 'of'], ['is', 'them'], ['is', 'very'], ['is', 'complex']], [['very', 'them'], ['very', 'is'], ['very', 'complex'], ['very', ',']], [['complex', 'is'], ['complex', 'very'], ['complex', ','], ['complex', 'but']], [[',', 'very'], [',', 'complex'], [',', 'but'], [',', 'the']], [['but', 'complex'], ['but', ','], ['but', 'the'], ['but', 'link']], [['the', ','], ['the', 'but'], ['the', 'link'], ['the', 'between']], [['link', 'but'], ['link', 'the'], ['link', 'between'], ['link', 'the']], [['between', 'the'], ['between', 'link'], ['between', 'the'], ['between', 'two']], [['the', 'link'], ['t

In [10]:
def make_input_batches(training_set, batch_size):

    sentences = training_set
    random.shuffle(sentences)
    
    new_data = []
    num_samples = len(sentences)
    for idx in range(num_samples // batch_size):
        batch = sentences[(idx)*batch_size : (idx+1)*batch_size]
        new_data.append(batch)
    
    return np.array(new_data)

In [57]:
def get_one_hot_vector(word, vocab_size=V):
    one_hot = torch.zeros(vocab_size)
    one_hot[w2i[word]] = 1.0

    return one_hot

In [11]:
BATCH_SIZE = 10
input_data = make_input_batches(training_data, BATCH_SIZE)

In [46]:
class linearity(nn.Module):
    def __init__(self, embedding_dimension, vocabulary_size):
        super(linearity, self).__init__()
        self.fc1 = nn.Linear(vocabulary_size, embedding_dimension)
    
    def forward(self, x):
        return self.fc1(x)

In [62]:
class BSG_Net(nn.Module):
    def __init__(self, vocabulary_size, embedding_dimension_1=20, embedding_dimension_2=60):
        super(BSG_Net, self).__init__()
        self.fc1 = nn.Linear(vocabulary_size, embedding_dimension_1)
        self.fc2 = nn.Linear(embedding_dimension_1 * 2, embedding_dimension_2)


        self.E_D1 = embedding_dimension_1
        self.E_D2 = embedding_dimension_2
        
    def forward(self, x):

        context_representation = torch.zeros(self.E_D2)

        for pair in x:
            center_word = self.fc1(get_one_hot_vector(pair[0]))
            context_word = self.fc2(get_one_hot_vector(pair[1]))

            concatenated = torch.cat([center_word, context_word], dim=0)
            concatenated = F.relu(model_2(concatenated))
            context_representation += concatenated
        
        return context_representation


In [63]:
EMBEDDING_DIMENSION = 20
EMBEDDING_DIMENSION_2 = 60

model = BSG_Net(V, EMBEDDING_DIMENSION, EMBEDDING_DIMENSION_2)
X_data = []

for sentence in context_data:
    for context_set in sentence:
        print(model(context_set))
        break
    break
    
    
    
#     for context_set in sentence:
#         to_add = []
#         for pair in context_set:
#             center_word = model_1(get_one_hot_vector(pair[0]))
#             context_word = model_1(get_one_hot_vector(pair[1]))

#             concatenated = torch.cat([center_word, context_word], dim=0)
#             concatenated = F.relu(model_2(concatenated))
#             to_add.append(concatenated)
            
#         X_data.append(to_add)

RuntimeError: size mismatch, m1: [1 x 74], m2: [40 x 60] at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/TH/generic/THTensorMath.c:2033

In [56]:
X_data[0]

[tensor([ 0.0680,  0.0000,  0.0074,  0.0000,  0.0726,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0328,
          0.0481,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0672,  0.1533,  0.1062,  0.0000,  0.0000,  0.0333,
          0.0000,  0.0000,  0.0000,  0.1562,  0.1944,  0.0000,  0.0000,
          0.1469,  0.0000,  0.0000,  0.0000,  0.0485,  0.0000,  0.1950,
          0.0022,  0.0000,  0.0000,  0.0000,  0.0531,  0.0259,  0.0000,
          0.0000,  0.1091,  0.0000,  0.0505,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.1352]),
 tensor([ 0.0766,  0.0000,  0.0428,  0.0000,  0.0813,  0.0158,  0.0000,
          0.0816,  0.0000,  0.0000,  0.0047,  0.0025,  0.0000,  0.0000,
          0.0110,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0975,  0.1702,  0.1671,  0.0000,  0.0000,  0.0423,
          0.0605,  0.0000,  0.0000,  0.1550,  0.1246,  0.0067,  0.0000,
          0.1545,